In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import argparse
import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

2025-03-04 12:39:34.859357: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-04 12:39:34.876854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-04 12:39:34.897296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-04 12:39:34.902361: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 12:39:34.917130: I tensorflow/core/platform/cpu_feature_guar

In [3]:
batch_size = 16
model_name_or_path = "roberta-large"
task = "mnli"
peft_type = PeftType.LORA
device = "cuda"
num_epochs = 20

In [4]:
peft_config = LoraConfig(use_dora=True, task_type="SEQ_CLS", inference_mode=False, r=4, lora_alpha=16, lora_dropout=0.1)
lr = 3e-4

In [5]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

datasets = load_dataset("glue", task)
metric = evaluate.load("glue", task)


def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples['premise'], examples['hypothesis'], truncation=True, max_length=None)
    return outputs


tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "premise", "hypothesis"],
)

# We also rename the 'label' column to 'labels' which is the expected name for labels by the models of the
# transformers library
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["validation_matched"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)
mismatch_dataloader = DataLoader(
    tokenized_datasets["validation_mismatched"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)
# test_dataloader = DataLoader(
#     tokenized_datasets["test"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
# )

Using the latest cached version of the module from /home/qula0496/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--glue/05234ba7acc44554edcca0978db5fa3bc600eeee66229abe79ff9887eacaf3ed (last modified on Tue Oct 22 15:33:12 2024) since it couldn't be found locally at evaluate-metric--glue, or remotely on the Hugging Face Hub.


In [6]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, num_labels=3, return_dict=True)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,495,043 || all params: 356,857,862 || trainable%: 0.4189


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [8]:
print_trainable_parameters(model)

trainable params: 1495043 || all params: 356857862 || trainable%: 0.42


In [9]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [10]:
from modules.clientFeDoRA_KD import *

In [11]:
from torch.optim import AdamW
# Create clients (e.g., 3 clients)
num_clients = 5

# Get the total length of the dataset
total_len = len(tokenized_datasets["train"])

# Calculate the number of samples per client and the remainder
split_size = total_len // num_clients
remainder = total_len % num_clients

# Create list for splits
split_lengths = [split_size] * num_clients

# Distribute the remainder among the first few clients
for i in range(remainder):
    split_lengths[i] += 1

# Perform the split
# client_data_splits = torch.utils.data.random_split(train_dataloader, split_lengths)
train_dataloader = torch.utils.data.random_split(tokenized_datasets["train"], split_lengths)
client_data_splits = [DataLoader(train_dataloader[i], shuffle=True, collate_fn=collate_fn, batch_size=batch_size) for i in range(num_clients)]
clients = [ClientFeDoRA_KD(client_id=i, model=copy.deepcopy(model), data=client_data_splits[i], device=device) for i in range(num_clients)]

In [12]:
from modules.server import *
server = Server(global_model=model, device=device)

In [13]:
import wandb
wandb.init(project="FeDORA Project - MNLI", entity="quanla", name="FeDoRA")

03/04/2025 12:39:58:ERROR:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [14]:
# 6. Federated Training Loop
num_rounds = 5  # Number of federated learning rounds
epochs_per_client = 5  # Number of epochs per client in each round

for round_num in range(num_rounds):
    print(f"\n--- Federated Learning Round {round_num + 1} ---")
    
    client_models = []
    
    # Train each client locally
    for client in clients:
        print(f"\nTraining Client {client.client_id}")
        client.train(epochs=epochs_per_client, regularization_strength=1.0, base_model=model)
        client_models.append(client.get_parameters())

    # Aggregate client models on the server
    print("\nAggregating client models on the server...")
    server.aggregate(client_models)
    eval_metric = server.evaluate(eval_dataloader, metric=metric)
    print(eval_metric)
    wandb.log(eval_metric)
    # evaluate(lora_model, val_dataset)

    
    # Update each client's model with the new global model
    for client in clients:
        client.set_parameters(server.global_model.state_dict())

wandb.finish()


--- Federated Learning Round 1 ---

Training Client 0


 22%|██▏       | 1072/4909 [04:41<16:47,  3.81it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 23.65 GiB of which 12.12 MiB is free. Process 1815202 has 534.00 MiB memory in use. Including non-PyTorch memory, this process has 23.10 GiB memory in use. Of the allocated memory 20.02 GiB is allocated by PyTorch, and 2.62 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# model.to(device)
# for epoch in range(num_epochs):
#     model.train()
#     for step, batch in enumerate(tqdm(train_dataloader)):
#         batch.to(device)
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()

#     model.eval()
#     for step, batch in enumerate(tqdm(eval_dataloader)):
#         batch.to(device)
#         with torch.no_grad():
#             outputs = model(**batch)
#         predictions = outputs.logits.argmax(dim=-1)
#         predictions, references = predictions, batch["labels"]
#         metric.add_batch(
#             predictions=predictions,
#             references=references,
#         )

#     eval_metric = metric.compute()
#     print(f"epoch {epoch}:", eval_metric)

In [ ]:
eval_metric = server.evaluate(mismatch_dataloader, metric=metric)
print(eval_metric)

  0%|          | 0/615 [00:00<?, ?it/s]

100%|██████████| 615/615 [00:21<00:00, 28.02it/s]

{'accuracy': 0.8927990235964198}
